In [1]:

from wikidata.client import Client
import itertools
from os import listdir
from os.path import isfile, join
from os import path
import requests
import json
import os
from datetime import date
import nltk
import pandas as pd
import re
import datetime
import random
from itertools import repeat

In [2]:
class Knowledge_extract(object): 
    value=['id','time','text']
    existing_ent=[]
    
    @staticmethod
    def get_entities(directory='./tmp/'):
        if path.exists(directory):
            jsonfiles = [f for f in listdir(directory) if isfile(join(directory, f))]
            Knowledge_extract.existing_ent=[f.split('.')[0] for f in jsonfiles]
            return Knowledge_extract.existing_ent

    def __init__(self, entityId, propsIds=[]):
        self.entityId = entityId
        self.propsIds = propsIds
        self.flag=False
        if len(Knowledge_extract.existing_ent)==0:
            Knowledge_extract.get_entities()
        if self.entityId not in Knowledge_extract.existing_ent:
            self.client=Client()
            self.entity=self.client.get(entityId, load=True)
            self.eid2v={}
            self.pid2v={}
            self.eid2alt={}
            self.result={}
            self.extract()
            self.alt_label_modification()
        else:
            #print('entity already exist')
            self.flag=True
    
    def extract(self):
        if len(self.propsIds) > 0:
            keys=(key for key in self.entity.keys() if key.id in self.propsIds)
        else:
            keys=self.entity.keys()
            
        dct = {}
        for key in keys:
            #print(key.id+" "+str(key.label))
            if key.id not in self.pid2v:
                self.pid2v[key.id]=str(key.label)
            tmp=[]
            for claim in self.entity.data['claims'][key.id]:
                temp1={}
                try:
                    if type(claim['mainsnak'].get('datavalue').get('value'))==str:
                        tmp.append(claim['mainsnak']['datavalue']['value'])
                    else:
                        avail=[i for i in Knowledge_extract.value if i in claim['mainsnak']['datavalue']['value'].keys()]
                        if len(avail) > 0:
                            if avail[0]=='id':
                                if claim.get('qualifiers') is not None:
                                    temp={}
                                    for quals in claim['qualifiers']:
                                        lbl=str(self.client.get(quals).label)
                                        if quals not in self.pid2v:
                                            self.pid2v[quals]=lbl
                                        q_val=[]
                                        for qual in claim['qualifiers'][quals]:
                                            #print(qual.get('datavalue'))
                                            if qual.get('datavalue') is not None:
                                                if type(qual.get('datavalue').get('value'))==str:
                                                    q_val.append(qual['datavalue']['value'])
                                                    #temp[lbl]=qual['datavalue']['value']
                                                else:
                                                    avail_q=[i for i in Knowledge_extract.value if i in qual['datavalue']['value'].keys()]
                                                    #print(avail_q)
                                                    if len(avail_q) > 0:
                                                        if avail_q[0]=='id':
                                                            ent=qual['datavalue']['value'][avail_q[0]]
                                                            ent_lbl=self.client.get(qual['datavalue']['value'][avail_q[0]])
                                                            if ent not in self.eid2v:
                                                                self.eid2v[ent]=str(ent_lbl.label)
                                                            if ent not in self.eid2alt:
                                                                self.eid2alt[ent]=ent_lbl.attributes.get('aliases').get('en')
                                                            q_val.append(str(ent_lbl.label))
                                                            #temp[lbl]=str(ent_lbl.label)
                                                        else:
                                                            q_val.append(qual['datavalue']['value'][avail_q[0]])
                                                            #temp[lbl]=qual['datavalue']['value'][avail_q[0]]
                                                    else:
                                                        continue
                                            else:
                                                continue
                                        if len(q_val) > 0:
                                            temp[lbl]=q_val
                                    ent1=claim['mainsnak']['datavalue']['value'].get(avail[0])
                                    ent1_lbl=str(self.client.get(claim['mainsnak']['datavalue']['value'].get(avail[0])).label)
                                    if ent1 not in self.eid2v:
                                        self.eid2v[ent1]=ent1_lbl
                                    temp1[ent1_lbl]=temp
                                if bool(temp1):
                                    tmp.append(temp1)
                                else:
                                    ent_lbl2=self.client.get(claim['mainsnak']['datavalue']['value'].get(avail[0]))
                                    ent2=claim['mainsnak']['datavalue']['value'].get(avail[0])
                                    if ent2 not in self.eid2v:
                                        self.eid2v[ent2]=str(ent_lbl2.label)
                                    if ent2 not in self.eid2alt:
                                        self.eid2alt[ent2]=ent_lbl2.attributes.get('aliases').get('en')
                                    tmp.append(str(ent_lbl2.label))
                            else:
                                tmp.append(claim['mainsnak']['datavalue']['value'].get(avail[0]))
                        else:
                            continue
                    dct[str(key.label)]=tmp
                except AttributeError as ae:
                    continue
        if self.entity.id not in self.eid2v:
            self.eid2v[self.entity.id]=str(self.entity.label)
        if self.entity.id not in self.eid2alt:
            self.eid2alt[self.entity.id]=self.entity.attributes.get('aliases').get('en')
        self.result[str(self.entity.label)]=dct
        self.result['alt_label']=self.eid2alt
        self.result['entity']=self.eid2v
        self.result['prop']=self.pid2v
        return
    
    def get_result(self):
            return self.result
        
    def alt_label_modification(self):
        for ent in self.eid2alt:
            if self.eid2alt[ent] is not None:
                self.eid2alt[ent]=[a['value'] for a in self.eid2alt[ent]]
            else:
                continue
            
    def ent2value(self):
        return self.eid2v

    def prop2value(self):
        return self.pid2v

    def ent2alt(self):
        return self.eid2alt

    def entlabel(self):
        return str(self.entity.label)
    
    def write_json(self, data=None, filename='', directory='./tmp/'):
        if not os.path.exists(directory):
            try:
                os.makedirs(directory)
            except OSError as e:
                if e.errno != errno.EEXIST:
                    raise
        if filename=='':
            filename=str(self.entityId)
        else:
            filename=filename
        path=os.path.join(directory, filename + "." + 'json')
        if data is None:
            with open(path, 'w') as fp:
                json.dump(self.result, fp, sort_keys=True, indent=4)
        else:
            with open(path, 'w') as fp:
                json.dump(data, fp, sort_keys=True, indent=4)
            
def get_entity_id(query,api="https://www.wikidata.org/w/api.php"):
    params={
    'action':'wbsearchentities',
    'search':query,
    'language':'en',
    'format':'json'
    }
    ent=requests.get(api, params=params)
    #print(ent.json()['search'])
    if len(ent.json()['search']) > 0:
        return ent.json()['search'][0]['id']
    else:
        return None

In [3]:
def load_json(filename, directory='./tmp/'):
    path=os.path.join(directory, filename + "." + 'json')
    with open(path) as f:
        data = json.load(f)
    return data

In [4]:
def write_file(data, filename, location='./'):
    path=os.path.join(location, filename + "." + 'json')
    with open(path,'w') as f:
        json.dump(data, f, sort_keys=True, indent=4)
    print('Done')
    return

In [5]:
def check_values(values, prop, directory='./tmp/'):#return all the entity whose prop value in values
    #(i.e occupation which are not related to cs)
    ent={}
    ent_id=[]
    ent_lbls=labels_dict(directory,flag='ent')
    p_lbls=labels_dict(directory,flag='prop')
    lbl=p_lbls[prop]
    if path.exists(directory):
        jsonfiles = (f for f in listdir(directory) if isfile(join(directory, f)) and f.endswith('json'))
        for jfile in jsonfiles:
            entid=jfile.split('.')[0]
            file=join(directory, jfile)
            with open(file) as f:
                data = json.load(f)
                name=list(data.keys())[0]
                ent_data=data[name]
                if lbl in ent_data.keys():
                    prop_data=ent_data[lbl]
                    for a in prop_data:
                        if type(a)==str:
                            if a in values:
                                ent[name]=prop_data
                                ent_id.append(entid)
                                break
                        else:
                            if list(a.keys())[0] in values:
                                ent[name]=prop_data
                                ent_id.append(entid)
                                break
    return ent, ent_id  

In [6]:
def labels_dict(directory='./tmp/',flag='ent'):#combine all dictionary in entity files related to ent2v or
    #pid2v or ent2alt based on flag argument.
    flag=flag.lower()
    labels={}
    if path.exists(directory):
        jsonfiles = (f for f in listdir(directory) if isfile(join(directory, f)) and f.endswith('json'))
        for jfile in jsonfiles:
            #print(jfile)
            file=join(directory, jfile)
            with open(file) as f:
                data=json.load(f)
                if flag.startswith('ent'):
                    labels={**labels, **data.get('entity')} 
                elif flag.startswith('prop'):
                    labels={**labels, **data.get('prop')}
                elif flag.startswith('alt'):
                    labels={**labels, **data.get('alt_label')}
                else:
                    print("Please provide correct flag value for labels: item label(ent),property label(prop) or aliase(alt),default='ent'")
    return labels

In [7]:
%%time
occupation=load_json('occcupation', directory='./')
dictt, entity_ids=check_values(occupation,'P106')
people=pd.read_csv('./neurotree/people_cs.tsv', sep='\t', encoding='utf-8',  escapechar='\\')
connect=pd.read_csv('./neurotree/connect_cs.tsv', sep='\t', encoding='latin-1',  escapechar='\\')
people['name']=people['firstname'].fillna('')+" "+people['middlename'].fillna('')+" "+people['lastname'].fillna('')
#people['name']=people['name'].str.replace('.','',regex=False)
people['name']=people['name'].str.replace(' +',' ',regex=True)
eid2label_dict=labels_dict(directory='./tmp/',flag='ent')
eid2alias_dict=labels_dict(directory='./tmp/',flag='alt')
def find_similar(name):
    global people
    global eid2label_dict
    global eid2alias_dict
    ent_id=list(eid2label_dict.keys())[list(eid2label_dict.values()).index(name)]
    if eid2alias_dict.get(ent_id) is not None:
        aliases=eid2alias_dict.get(ent_id) 
    else: 
        aliases=list()
    aliases.append(name)
    names=list(reversed(aliases))
    names=[name.strip() for name in names]
    names=[name.replace('.','') for name in names]
    names=[re.sub(' +',' ',name) for name in names]
    sim=people['name'].str.contains('|'.join(names), flags=re.IGNORECASE, regex=True)
    sim_inx=sim[sim==True].index
    if len(sim_inx)==1:
        pid=str(people.values[sim_inx[0]][0])
        print("pid : %s" %pid)
        print(name+' Information Available in Academic tree')
        return False
    else:
        return True

CPU times: user 2.87 s, sys: 177 ms, total: 3.05 s
Wall time: 3.41 s


In [8]:
def create_inst(entity):
    props=['P27','P106','P101','P184','P185','P1026','P802','P1066','P800','P856','P2456','P2381','P864',
       'P108','P69','P512','P166','P463','P39','P496']
    try:
        inst=Knowledge_extract(entity, props)
        #print(inst.flag)
        if not inst.flag:
            inst.write_json(directory='./tmp/')
        return inst.result
    except Exception as e:
        print("not created inst(exception occured)"+entity)
        return None
def entity_info(name):
    global entity_IdInDir
    global entity_list
    ent_id=list(eid2label_dict.keys())[list(eid2label_dict.values()).index(name)]
    if ent_id not in entity_list:
        try:
            ent_info=load_json(ent_id, directory='./tmp/')
        except FileNotFoundError as fnf:
            print('file not found, Creating....')
            ent_info=create_inst(ent_id)
            entity_IdInDir.append(ent_id)
        return (ent_info, ent_id)
    else:
        print("Entity Already Added")
        return None

In [9]:
def split_name(name):
    name=name.split()
    if len(name)==3:
        firstname=name[0]
        middlename=name[1]
        lastname=name[2]
    elif len(name)==2:
        firstname=name[0]
        middlename=""
        lastname=name[1]
    elif len(name)==1:
        firstname=name[0]
        middlename=""
        lastname=""
    else:
        firstname=name[0]
        middlename=name[1:-1]
        lastname=name[-1]
    return (firstname, middlename, lastname)

In [10]:
'''props=['P27','P106','P101','P184','P185','P1026','P802','P1066','P800','P856','P2456','P2381','P864',
       'P108','P69','P512','P166','P463','P39','P496']
aln=Knowledge_extract('Q15637844',props)'''

"props=['P27','P106','P101','P184','P185','P1026','P802','P1066','P800','P856','P2456','P2381','P864',\n       'P108','P69','P512','P166','P463','P39','P496']\naln=Knowledge_extract('Q15637844',props)"

In [11]:
#a=entity_info('William Boone')

In [12]:
def property_value(val):
    if val is not None:
        prop=set()
        for a in val:
            if type(a)==str:
                prop.add(a)
            else:
                prop.add("".join(a.keys()))
        return prop
    else:
        return list()

In [18]:
def mapping_info(ent_info, entt_id):
    #global wiki_researcher
    name=list(ent_info.keys())[0]
    ent_info=ent_info[name]
    info=[]
    #temp=[]
    tmp_str=" ".join(property_value(ent_info.get('occupation'))) 
    if entt_id not in entity_ids or 'computer' in tmp_str or 'mathematician' in tmp_str:
        if ent_info.get('Academic Tree ID') is not None:
            pid="".join(ent_info.get('Academic Tree ID'))
        else:
            #pid=max(people['pid'])+1
            pid=entt_id
        info.append(pid)         #this need to be replaced with pid 
        name_tup=split_name(name)
        info.append(name_tup[0])
        info.append(name_tup[1])
        info.append(name_tup[2])
        info.append('degree')
        info.append(";".join(property_value(ent_info.get('employer'))))
        info.append('locid')
        info.append(";".join(property_value(ent_info.get('field of work') 
                                            if ent_info.get('field of work') is not None else ent_info.get('occupation'))))
        info.append('major_area')
        info.append(";".join(property_value(ent_info.get('award received'))))
        info.append("".join(ent_info.get('ORCID iD')) if ent_info.get('ORCID iD') is not None else '')
        info.append("".join(ent_info.get('official website')) if ent_info.get('official website') is not None else '')
        info.append('Wikidata')
        info.append(datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
        #temp.append(entt_id)
        #print(name+" "+'Added')
        #wiki_researcher.append(info)
        return (info, entt_id)
    else:
        print(name+" "+'occupation not belong to computer science......')
    return None

In [19]:
def prop_value(entity, props=['P184','P185','P802','P1066'], directory='./tmp/',qual=False): #return all entity for property supervisor, 
    #supervisor_of,student, student_of
    values=[]
    props_lbl=[]
    path=os.path.join(directory, entity + "." + 'json')
    if isfile(path):
        if props[0].startswith('P'):
            p_lbls=labels_dict(directory,flag='prop')
            for p in props:
                props_lbl.append(p_lbls[p])
        else:
            props_lbl=props
        with open(path) as f:
            data = json.load(f)
        tmp_data=data[list(data.keys())[0]]
        keys=[lbl for lbl in props_lbl if lbl in tmp_data.keys()]
        if qual:
            for key in keys:
                values.append(tmp_data[key])
        else:
            for key in keys:
                for val in tmp_data[key]:
                    if type(val)==str:
                        values.append(val)
                    else:
                        values.append(list(val.keys())[0])
        return set(values)
    else:
        print('entity not found')
        return None 

In [20]:
def read_entity_IdInDir(directory='./tmp/'):
    jsonfiles = [f for f in listdir(directory) if isfile(join(directory, f))]
    entity=[f.split('.')[0] for f in jsonfiles]
    return entity

In [21]:
'''t_conn=[]
wiki_researcher=[]'''

't_conn=[]\nwiki_researcher=[]'

In [22]:
'''e='Q7917308'
obj=prop_value(e)
temp=[]
if obj is not None:
    for name in obj:
        if find_similar(name):
            ent_info, entt_id=entity_info(name)
            if ent_info is not None:
                mapping_info(ent_info, entt_id)
            else:
                continue
t=list(zip(repeat(e),temp))
t_conn=t_conn + t'''

"e='Q7917308'\nobj=prop_value(e)\ntemp=[]\nif obj is not None:\n    for name in obj:\n        if find_similar(name):\n            ent_info, entt_id=entity_info(name)\n            if ent_info is not None:\n                mapping_info(ent_info, entt_id)\n            else:\n                continue\nt=list(zip(repeat(e),temp))\nt_conn=t_conn + t"

In [23]:
#write_file(entity_IdInDir,'total_entity_in_tmp')

In [24]:
#entity_IdInDir=read_entity_IdInDir(directory='./tmp/')

In [25]:
%%time
def wikidata_add(directory='./tmp/'):
    entity_IdInDir=read_entity_IdInDir(directory=directory)
    conn=[]
    wiki_researcher=[]
    entity_list=[]
    for e in entity_IdInDir:
        if e not in entity_list and e not in entity_ids:
            print("entity:%s" %e)
            obj=prop_value(e)
            print("supervisor/student:%d"%len(obj))
            temp=[]
            if obj is not None:
                for name in obj:
                    if find_similar(name):
                        ent_info_entt_id=entity_info(name)
                        if ent_info_entt_id is not None:
                            info_entid=mapping_info(ent_info_entt_id[0], ent_info_entt_id[1])
                            if info_entid is not None:
                                temp.append(info_entid[1])
                                wiki_researcher.append(info_entid[0])   
                            else:
                                continue
                        else:
                            print('Not required information is available for entity id:%s'%entt_id)
                            continue
            t=list(zip(repeat(e),temp))
            conn=conn + t
            entity_list.append(e)
        else:
            print('Entity already addded Or entity occupation not belongs to cs')
            continue
    return wiki_researcher, conn, entity_list

entity:Q42864989
supervisor/student:0
entity:Q7179589
supervisor/student:0
entity:Q42599771
supervisor/student:0
entity:Q45917159
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
Entity already addded Or entity occupation not belongs to cs
entity:Q1771279
supervisor/student:2
entity:Q92806
supervisor/student:1
entity:Q63126537
supervisor/student:0
entity:Q26923147
supervisor/student:1
Entity already addded Or entity occupation not belongs to cs
entity:Q572680
supervisor/student:1
pid : 743456
Azriel Lévy Information Available in Academic tree
entity:Q56166797
supervisor/student:0
entity:Q982690
supervisor/student:5
Entity already addded Or entity occupation not belongs to cs
Entity already addded Or entity occupation not belongs to cs
entity:Q22005497
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q441185
supervisor/student:5
entity:Q62611844
supervisor/student:0
Entity already addded Or entity occupation not be

file not found, Creating....
entity:Q5369117
supervisor/student:2
entity:Q7905286
supervisor/student:1
entity:Q2039726
supervisor/student:1
entity:Q57097654
supervisor/student:0
entity:Q16062558
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
Entity already addded Or entity occupation not belongs to cs
entity:Q20675506
supervisor/student:0
entity:Q9912
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q15441442
supervisor/student:1
entity:Q40424579
supervisor/student:0
entity:Q58532228
supervisor/student:0
entity:Q1350952
supervisor/student:1
pid : 739942
Kurt Hensel Information Available in Academic tree
entity:Q7526
supervisor/student:2
pid : 416329
Gene Tsudik Information Available in Academic tree
entity:Q16734091
supervisor/student:0
entity:Q6929572
supervisor/student:0
entity:Q38800984
supervisor/student:0
entity:Q65350
supervisor/student:3
pid : 1932
Werner Heisenberg Information Available in Academic tree


supervisor/student:0
entity:Q7383152
supervisor/student:0
entity:Q21572913
supervisor/student:1
Entity already addded Or entity occupation not belongs to cs
entity:Q58335903
supervisor/student:0
entity:Q19657252
supervisor/student:0
entity:Q23069025
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
Entity already addded Or entity occupation not belongs to cs
entity:Q102214
supervisor/student:0
entity:Q61746864
supervisor/student:0
entity:Q63815290
supervisor/student:0
entity:Q62067809
supervisor/student:0
entity:Q28871854
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q5208432
supervisor/student:1
entity:Q24690775
supervisor/student:0
entity:Q21077813
supervisor/student:0
entity:Q5531206
supervisor/student:0
entity:Q59834725
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q1398945
supervisor/student:2
Entity Already Added
Not required information is available for entity id

supervisor/student:0
entity:Q7614423
supervisor/student:0
entity:Q841127
supervisor/student:1
entity:Q48503988
supervisor/student:0
entity:Q93109
supervisor/student:2
Entity Already Added
Not required information is available for entity id:None
pid : 677936
Seymour Papert Information Available in Academic tree
entity:Q16015301
supervisor/student:3
file not found, Creating....
file not found, Creating....
Entity already addded Or entity occupation not belongs to cs
Entity already addded Or entity occupation not belongs to cs
Entity already addded Or entity occupation not belongs to cs
entity:Q6265139
supervisor/student:1
Entity Already Added
Not required information is available for entity id:None
entity:Q45510443
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q4274128
supervisor/student:0
entity:Q64481799
supervisor/student:0
entity:Q450151
supervisor/student:2
file not found, Creating....
entity:Q92819
supervisor/student:2
pid : 416292
Bhubane

/home/dj/webScrapper/env/lib/python3.6/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


...

supervisor/student:0
entity:Q4054157
supervisor/student:1
pid : 184261
Umesh Vazirani Information Available in Academic tree
entity:Q48906639
supervisor/student:0
entity:Q59630318
supervisor/student:0
entity:Q7282986
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q790909
supervisor/student:0
entity:Q37828474
supervisor/student:0
entity:Q59661842
supervisor/student:0
entity:Q41805832
supervisor/student:1
Entity Already Added
Not required information is available for entity id:None
Entity already addded Or entity occupation not belongs to cs
Entity already addded Or entity occupation not belongs to cs
entity:Q528493
supervisor/student:2
pid : 48092
Alfred Tarski Information Available in Academic tree
Entity Already Added
Not required information is available for entity id:None
entity:Q61727910
supervisor/student:0
entity:Q17995988
supervisor/student:0
entity:Q57515849
supervisor/student:0
entity:Q57239897
supervisor/student:0
entity:Q5039998
supe

entity:Q7604112
supervisor/student:0
entity:Q44817036
supervisor/student:0
entity:Q1352633
supervisor/student:1
Entity Already Added
Not required information is available for entity id:None
entity:Q910303
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q48001653
supervisor/student:0
entity:Q4777216
supervisor/student:0
entity:Q6224420
supervisor/student:1
Entity Already Added
Not required information is available for entity id:None
entity:Q21535345
supervisor/student:0
entity:Q7284410
supervisor/student:1
Entity Already Added
Not required information is available for entity id:None
entity:Q56913312
supervisor/student:0
entity:Q59818474
supervisor/student:0
entity:Q6191950
supervisor/student:2
Entity Already Added
Not required information is available for entity id:None
pid : 370104
Larry Travis Information Available in Academic tree
entity:Q62563599
supervisor/student:0
entity:Q7364844
supervisor/student:2
file not found, Creating....
entity:Q46

supervisor/student:0
entity:Q56426441
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q45594855
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q45507739
supervisor/student:0
entity:Q1510203
supervisor/student:2
file not found, Creating....
Entity Already Added
Not required information is available for entity id:None
Entity already addded Or entity occupation not belongs to cs
entity:Q59698519
supervisor/student:0
entity:Q51120255
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q59684931
supervisor/student:0
entity:Q51140094
supervisor/student:0
entity:Q55091076
supervisor/student:0
entity:Q15059124
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q50286281
supervisor/student:0
entity:Q45441647
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
entity:Q30080329
supervisor/student:1
Entity Alread

supervisor/student:2
Entity Already Added
Not required information is available for entity id:None
Entity Already Added
Not required information is available for entity id:None
entity:Q5348683
supervisor/student:1
pid : 173139
Dana Angluin Information Available in Academic tree
entity:Q31995074
supervisor/student:0
entity:Q56842289
supervisor/student:1
Entity Already Added
Not required information is available for entity id:None
entity:Q91669
supervisor/student:7
file not found, Creating....
file not found, Creating....
Entity Already Added
Not required information is available for entity id:None
file not found, Creating....
entity:Q27916206
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
Entity already addded Or entity occupation not belongs to cs
entity:Q311958
supervisor/student:0
entity:Q7039821
supervisor/student:2
pid : 438329
Joseph Halpern Information Available in Academic tree
pid : 183365
Dana Pe'er Information Available in Academic tree
enti

supervisor/student:0
entity:Q2965679
supervisor/student:0
entity:Q11686192
supervisor/student:0
entity:Q30501612
supervisor/student:0
entity:Q42914963
supervisor/student:0
Entity already addded Or entity occupation not belongs to cs
Entity already addded Or entity occupation not belongs to cs
entity:Q3158400
supervisor/student:0
entity:Q37392413
supervisor/student:0
entity:Q45708890
supervisor/student:0
entity:Q56791563
supervisor/student:0
entity:Q62088637
supervisor/student:0
entity:Q4922
supervisor/student:1
Entity Already Added
Not required information is available for entity id:None
entity:Q1369818
supervisor/student:1
Entity Already Added
Not required information is available for entity id:None
entity:Q29340761
supervisor/student:0
entity:Q17003841
supervisor/student:0
entity:Q29452454
supervisor/student:0
entity:Q44651389
supervisor/student:0
entity:Q4339024
supervisor/student:0
entity:Q92765
supervisor/student:1
Entity Already Added
Not required information is available for ent

In [32]:
len(conn)

1552

In [35]:
count=0
for w in wiki_researcher:
    if 'Q' in w[0]:
        count+=1
print(count)

1027


In [34]:
people[people['pid']==168048]

,pid,firstname,middlename,lastname,degrees,location,locid,area,majorarea,award,orcid_id,homepage,addedby,dateadded,name
474,168048,Andy,NaN,van Dam,NaN,Brown University,30,Graphics and Visualization,computerscience,NaN,NaN,NaN,david,2015-12-23 21:44:03,Andy van Dam


In [30]:
people[people['name'].str.contains("Andries",flags=re.IGNORECASE, regex=True)]

,pid,firstname,middlename,lastname,degrees,location,locid,area,majorarea,award,orcid_id,homepage,addedby,dateadded,name
4643,416582,Andries,v.,Dam,N,Brown University,30,Computer Science,computerscience,N,N,NaN,pq,2016-03-24 15:31:24,Andries v. Dam
21153,685893,Louis,NaN,Andriessen,NaN,Royal Conservatorium of The Hague,25881,NaN,music,NaN,NaN,NaN,hugendubler2,2016-10-22 17:52:31,Louis Andriessen


In [180]:
'''if not aln.flag:
    aln.write_json()'''

'if not aln.flag:\n    aln.write_json()'

In [104]:
#get_entity_id("https://www.wikidata.org/w/api.php",'Alonzo Church')

In [12]:
def allValuesForProp(prop, directory='./tmp/'):#return all distinct values for given property
    values=[]
    p_lbls=labels_dict(directory, flag='prop')
    lbl=p_lbls[prop]
    if path.exists(directory):
        jsonfiles = (f for f in listdir(directory) if isfile(join(directory, f)) and f.endswith('json'))
        for jfile in jsonfiles:
            file=join(directory, jfile)
            with open(file) as f:
                data = json.load(f)
                ent_data=data[list(data.keys())[0]]
                if lbl in ent_data.keys():
                    prop_data=ent_data[lbl]
                    for a in prop_data:
                        if type(a)==str:
                            values.append(a)
                        else:
                            values.append(list(a.keys())[0])
                else:
                    continue
    return set(values)     

In [9]:
#values_p=allValuesForProp('P106')

In [97]:
occupation=['American football player','Australian rules footballer','Canadian football player', 
            'Catholic priest', 'Esperantist', 'Patriota', 'Sepoy', 'actor', 'animator', 'announcer', 'artisan',
            'artist', 'association football manager', 'association football player', 'athlete',
            'athletics competitor', 'badminton player', 'banker', 'baseball player', 'beach volleyball player',
            'biathlete', 'boxer', 'bridge player', 'caricaturist', 'character actor', 'chef', 'chess composer', 
            'chess player', 'cinematographer', 'comedian', 'composer', 'conductor', 'cook', 'cricketer', 
            'criminologist', 'critic', 'dancer', 'designer', 'director', 'disc jockey', 'drummer', 'dub actor', 
            'ecologist', 'feminist', 'fencer', 'field hockey player', 'figure skater', 'film actor', 
            'film director', 'film editor', 'film producer', 'fonctionnaire de catégorie C', 'gangster', 
            'guitarist', 'gymnast', 'handball player', 'harpsichordist', 'ice hockey player', 'ichthyologist', 
            'instrumentalist', 'jazz musician', 'jockey', 'judge', 'judoka', 'jurist', 'kayaker', 'lawyer','lyricist', 
            'magician', 'make-up artist', 'model', 'monarch', 'motorcycle racer', 'mountaineer', 'musher', 'music arranger',
            'music pedagogue', 'musician', 'musicologist', 'naturalist', 'novelist', 'opinion journalist', 'painter', 
            'paleontologist', 'parson', 'pastor', 'pediatrician', 'percussionist', 'pianist', 'poet', 
            'political activist', 'political commissar', 'prince', 'producer', 'professional wrestler', 
            'pundit', 'racing automobile driver', 'racing driver', 'record producer','rower', 
            'rugby league player', 'rugby union player', 'screenwriter', 'sculptor', 'violinist', 
            'visual artist', 'voice actor', 'volleyball player', 'weightlifter','television actor', 
            'television producer', 'tennis player', 'theater director', 'theologian', 'trade unionist', 
            'translator', 'truck driver', 'serial killer', 'singer', 'singer-songwriter', 'skipper', 
            'snowboarder', 'songwriter', 'speed skater', 'sport cyclist', 'squash player', 'stage actor', 
            'suffragette', 'suffragist', 'swimmer', 'table tennis player', 'taekwondo athlete', 'tattoo artist', 
            'television actor', 'television producer', 'tennis player', 'theater director']